In [1]:
import pandas as pd
import networkx as nx
import numpy as np
import math
import os
import community

In [2]:
def average_degree(G):
    return sum(dict(G.degree()).values()) / len(G)

def numeratore_rLL(G):
    degree = dict(G.degree())
    somma=0
    avg = average_degree(G)
    for i in G.nodes() :
        for j in G.nodes() :
            if i < j:
                somma+=((degree[i] - avg)*(degree[j] - avg))
    return somma

def denominatore_rLL(G):
    degree = dict(G.degree())
    somma1=0
    somma2=0
    avg = average_degree(G)
    for i in G.nodes() :
        for j in G.nodes() :
            if i < j:
                somma1+=(degree[i] - avg)**2

    for i in G.nodes() :
        for j in G.nodes() :
            if i < j:
                somma2+=(degree[j] - avg)**2

    return math.sqrt(somma1*somma2)



def numeratore_rL1L2(G1,G2):
    degree1 = dict(G1.degree())
    degree2 = dict(G2.degree())
    somma=0
    avg1 = average_degree(G1)
    avg2 = average_degree(G2)
    for i in G1.nodes():
        
        for j in G2.nodes():
            somma+=(degree1[i] - avg1)-(degree2[j] - avg2)
    
    return somma

def denominatore_rL1L2(G1,G2):
    degree1 = dict(G1.degree())
    degree2 = dict(G2.degree())
    somma1=0
    somma2=0
    avg1 = average_degree(G1)
    avg2 = average_degree(G2)
    for i in G1.nodes():
        somma1=(degree1[i] - avg1)**2
    
    for j in G2.nodes():
        somma2+=(degree2[j] - avg2)**2

    return math.sqrt(somma1*somma2)

### degree-degree cross-correlation

single layer

In [ ]:
df_r= pd.DataFrame(columns = ['Date','Author Layer','Comment Layer','Topic Layer'])

# assign directory
directory = 'Graphs'

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    #checking if it is a file
    if os.path.isfile(f):
        print(filename[:11])
        G = nx.read_gexf(f)
        
        nodes=nx.get_node_attributes(G, 'color')

        topic_nodes=set()
        for u,v in nodes.items():
            if v=='green': 
                topic_nodes.add(u)
        #print(topic_nodes)

        author_nodes=set()
        for u,v in nodes.items():
            if v=='red': 
                author_nodes.add(u)
        #print(G.subgraph(author_nodes))

        comments_nodes=set()
        for u,v in nodes.items():
            if v=='blue': 
                comments_nodes.add(u)
        
        Layer_autori = G.subgraph(author_nodes)
        Layer_commenti = G.subgraph(comments_nodes)
        Layer_topic = G.subgraph(topic_nodes)
        
        num_a = numeratore_rLL(Layer_autori)
        
        den_a = denominatore_rLL(Layer_autori)
       
        if den_a == 0:
            r_a=0
        else:    
            r_a = num_a/den_a
        
        num_c = numeratore_rLL(Layer_commenti)
        
        den_c = denominatore_rLL(Layer_commenti)
        
        if den_c == 0:
            r_c = 0
        else:
            r_c = num_c/den_c

        num_t = numeratore_rLL(Layer_topic)
        
        den_t = denominatore_rLL(Layer_topic)
        
        if den_t == 0:
            r_t = 0
        else:
            r_t = num_t/den_t

        df_r = df_r.append({'Date':filename[:11],'Author Layer':r_a,'Comment Layer':r_c,'Topic Layer':r_t}, ignore_index = True)
        
        df_r['Date'] = pd.to_datetime(df_r['Date'])
        
        df_r.to_csv('rLL1.csv')


df_r.sort_values(by='Date', inplace = True)
df_r = df_r.set_index('Date')
df_r.to_csv('rLL1.csv')



multi-layer

In [ ]:
df_ra_rt= pd.DataFrame(columns = ['Date','Author - Topic'])

# assign directory
directory = 'Graphs'

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    #checking if it is a file
    if os.path.isfile(f):
        print(filename[:11])
        G = nx.read_gexf(f)
        
        nodes=nx.get_node_attributes(G, 'color')

        topic_nodes=set()
        for u,v in nodes.items():
            if v=='green': 
                topic_nodes.add(u)
        #print(topic_nodes)

        author_nodes=set()
        for u,v in nodes.items():
            if v=='red': 
                author_nodes.add(u)
        #print(G.subgraph(author_nodes))

        comments_nodes=set()
        for u,v in nodes.items():
            if v=='blue': 
                comments_nodes.add(u)
        
        Layer_autori = G.subgraph(author_nodes)
        #Layer_commenti = G.subgraph(comments_nodes)
        Layer_topic = G.subgraph(topic_nodes)
        
        num = numeratore_rL1L2(Layer_autori, Layer_topic)
        
        print(num)
        #r_c = nx.degree_pearson_correlation_coefficient(Layer_commenti)
        
        den = denominatore_rL1L2(Layer_autori, Layer_topic)
        
        if den == 0:
            a = 0
        else:    
            a = num / den
        print(a)
        df_ra_rt = df_ra_rt.append({'Date':filename[:11],'Author - Topic':a}, ignore_index = True)
        
        df_ra_rt['Date'] = pd.to_datetime(df_ra_rt['Date'])
        
        df_ra_rt.to_csv('assortativity_autori_topic1.csv')


df_ra_rt.sort_values(by='Date', inplace = True)
df_ra_rt = df_ra_rt.set_index('Date')
df_ra_rt.to_csv('assortativity_autori_topic1.csv')


In [ ]:
df_ra_rt= pd.DataFrame(columns = ['Date','Author - Comment'])

# assign directory
directory = 'Graphs'

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    #checking if it is a file
    if os.path.isfile(f):
        print(filename[:11])
        G = nx.read_gexf(f)
        
        nodes=nx.get_node_attributes(G, 'color')

        topic_nodes=set()
        for u,v in nodes.items():
            if v=='green': 
                topic_nodes.add(u)
        #print(topic_nodes)

        author_nodes=set()
        for u,v in nodes.items():
            if v=='red': 
                author_nodes.add(u)
        #print(G.subgraph(author_nodes))

        comments_nodes=set()
        for u,v in nodes.items():
            if v=='blue': 
                comments_nodes.add(u)
        
        Layer_autori = G.subgraph(author_nodes)
        Layer_commenti = G.subgraph(comments_nodes)
        #Layer_topic = G.subgraph(topic_nodes)
        
        num = numeratore_rL1L2(Layer_autori, Layer_commenti)
        
        print(num)
        #r_c = nx.degree_pearson_correlation_coefficient(Layer_commenti)
        
        den = denominatore_rL1L2(Layer_autori, Layer_commenti)
        
        if den == 0:
            a = 0
        else:    
            a = num / den
        print(a)
        df_ra_rt = df_ra_rt.append({'Date':filename[:11],'Author - Comment':a}, ignore_index = True)
        
        df_ra_rt['Date'] = pd.to_datetime(df_ra_rt['Date'])
        
        df_ra_rt.to_csv('assortativity_autori_commenti1.csv')


df_ra_rt.sort_values(by='Date', inplace = True)
df_ra_rt = df_ra_rt.set_index('Date')
df_ra_rt.to_csv('assortativity_autori_commenti1.csv')

In [ ]:
df_ra_rt= pd.DataFrame(columns = ['Date','Comment - Topic'])

# assign directory
directory = 'Graphs'

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    #checking if it is a file
    if os.path.isfile(f):
        print(filename[:11])
        G = nx.read_gexf(f)
        
        nodes=nx.get_node_attributes(G, 'color')

        topic_nodes=set()
        for u,v in nodes.items():
            if v=='green': 
                topic_nodes.add(u)
        #print(topic_nodes)

        author_nodes=set()
        for u,v in nodes.items():
            if v=='red': 
                author_nodes.add(u)
        #print(G.subgraph(author_nodes))

        comments_nodes=set()
        for u,v in nodes.items():
            if v=='blue': 
                comments_nodes.add(u)
        
        Layer_commenti = G.subgraph(comments_nodes)
        Layer_topic = G.subgraph(topic_nodes)
        
        num = numeratore_rL1L2( Layer_commenti, Layer_topic)
        
        print(num)
        #r_c = nx.degree_pearson_correlation_coefficient(Layer_commenti)
        
        den = denominatore_rL1L2(Layer_commenti, Layer_topic)
        
        if den == 0:
            a = 0
        else:    
            a = num / den
        print(a)
        df_ra_rt = df_ra_rt.append({'Date':filename[:11],'Comment - Topic':a}, ignore_index = True)
        
        df_ra_rt['Date'] = pd.to_datetime(df_ra_rt['Date'])
        
        df_ra_rt.to_csv('assortativity_commenti_topic1.csv')


df_ra_rt.sort_values(by='Date', inplace = True)
df_ra_rt = df_ra_rt.set_index('Date')
df_ra_rt.to_csv('assortativity_commenti_topic1.csv')

### Modularity

In [ ]:
df_modularity= pd.DataFrame(columns = ['Date','Author Layer','Comment Layer','Topic Layer','Multi-layer'])

# assign directory
directory = 'Graphs'
 
# iterate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    #checking if it is a file
    if os.path.isfile(f):
        print(filename[:11])
        G = nx.read_gexf(f)
        
        nodes=nx.get_node_attributes(G, 'color')

        topic_nodes=set()
        for u,v in nodes.items():
            if v=='green': 
                topic_nodes.add(u)
        #print(topic_nodes)

        author_nodes=set()
        for u,v in nodes.items():
            if v=='red': 
                author_nodes.add(u)
        #print(G.subgraph(author_nodes))

        comments_nodes=set()
        for u,v in nodes.items():
            if v=='blue': 
                comments_nodes.add(u)
        
        Layer_autori = G.subgraph(author_nodes)
        Layer_commenti = G.subgraph(comments_nodes)
        Layer_topic = G.subgraph(topic_nodes)
        try:
            a_part = nx.community.louvain_communities(Layer_autori, seed = 123)
            a_mod = nx.community.modularity(Layer_autori,a_part)
        except:
            a_mod = 0
            
        try:
            c_part = nx.community.louvain_communities(Layer_commenti, seed = 123)
            c_mod = nx.community.modularity(Layer_commenti,c_part)
        except:
            c_mod = 0
        
        try:
            t_part = nx.community.louvain_communities(Layer_topic, seed = 123)
            t_mod = nx.community.modularity(Layer_topic,t_part)
        except:
            t_mod = 0
        
        try:
            ml_part = nx.community.louvain_communities(G, weight='weight',seed = 123)
            ml_mod = nx.community.modularity(G, ml_part, weight='weight')
        except:
            ml_mod = 0
            
            
            
            
        df_modularity = df_modularity.append({'Date':filename[:11],'Author Layer':a_mod,'Comment Layer':c_mod,'Topic Layer':t_mod, 'Multi-layer': ml_mod}, ignore_index = True)
        

df_modularity['Date'] = pd.to_datetime(df_modularity['Date'])
df_modularity.sort_values(by='Date', inplace = True)
df_modularity = df_modularity.set_index('Date')
df_modularity.to_csv('modularity.csv')
    

### Average Shortest Path Lenght

In [ ]:

df_path= pd.DataFrame(columns = ['Date','Author Layer Avg Path','Topic Layer Avg Path'])
#df_path.to_csv('avg_path2.csv')
#df_path= pd.read_csv('avg_path.csv')
# assign directory
directory = 'Graphs'
 
# iterate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    #checking if it is a file
    if os.path.isfile(f):
        print(filename[:11])
        G = nx.read_gexf(f)
        
        nodes=nx.get_node_attributes(G, 'color')

        topic_nodes=set()
        for u,v in nodes.items():
            if v=='green': 
                topic_nodes.add(u)
        #print(topic_nodes)

        author_nodes=set()
        for u,v in nodes.items():
            if v=='red': 
                author_nodes.add(u)
        #print(G.subgraph(author_nodes))

        comments_nodes=set()
        for u,v in nodes.items():
            if v=='blue': 
                comments_nodes.add(u)
        
        Layer_autori = G.subgraph(author_nodes)
        Layer_commenti = G.subgraph(comments_nodes)
        Layer_topic = G.subgraph(topic_nodes)
        connected_autori = nx.connected_components(Layer_autori)
        
        connected_commenti = nx.connected_components(Layer_commenti)
        
        connected_topic = nx.connected_components(Layer_topic)
        a_path = []
        c_path = []
        t_path = []
        for cc in connected_autori:
            if len(cc) > 1:
                cc_aut = Layer_autori.subgraph(cc)
                
                a_path.append(nx.average_shortest_path_length(cc_aut))
        
        avg_a = sum(a_path)/len(a_path)
        
        
        #for cc in connected_commenti:
        #    if len(cc) > 1:
        #        cc_comm = Layer_commenti.subgraph(cc)
                
        #        c_path.append(nx.average_shortest_path_length(cc_comm))
        
        
        for cc in connected_topic:
            if len(cc) > 1:
                cc_topic = Layer_topic.subgraph(cc)
                
                t_path.append(nx.average_shortest_path_length(cc_topic))
        avg_t = sum(t_path)/len(t_path)
        
        #avg_c = sum(c_path)/len(c_path)
        
        
        df_path = df_path.append({'Date':filename[:11],'Author Layer Avg Path':avg_a,'Topic Layer Avg Path':avg_t}, ignore_index = True)
        
        df_path['Date'] = pd.to_datetime(df_path['Date'])
        df_path.to_csv('big_avg_path.csv')
        


df_path.sort_values(by='Date', inplace = True)
#df_path = df_path.set_index('Date')
df_path.to_csv('avg_path.csv')

### Clustering coefficient (transitivity)

In [ ]:
#transitivity
#df_cc= pd.DataFrame(columns = ['Date','Author Layer cc','Comment Layer cc','Topic Layer cc'])
df_cc= pd.DataFrame(columns = ['Date','Author Layer cc'])
#df_cc = pd.read_csv('transitivity.csv')
# assign directory
directory = 'Graphs'
 
# iterate over files in
# that directory
print(1)
for filename in os.listdir(directory):
    
    f = os.path.join(directory, filename)
    #checking if it is a file
    if os.path.isfile(f):
        print(filename[:11])
        G = nx.read_gexf(f)

        nodes=nx.get_node_attributes(G, 'color')

        topic_nodes=set()
        for u,v in nodes.items():
            if v=='green': 
                topic_nodes.add(u)
        #print(topic_nodes)

        author_nodes=set()
        for u,v in nodes.items():
            if v=='red': 
                author_nodes.add(u)
        #print(G.subgraph(author_nodes))

        comments_nodes=set()
        for u,v in nodes.items():
            if v=='blue': 
                comments_nodes.add(u)

        Layer_autori = G.subgraph(author_nodes)
        #Layer_commenti = G.subgraph(comments_nodes)
        #Layer_topic = G.subgraph(topic_nodes)

        #calcolo cc per ogni layer

        a_path = nx.transitivity(Layer_autori)
        print('author', a_path)
        df_cc = df_cc.append({'Date':filename[:11],'Author Layer cc':a_path}, ignore_index = True)
        

    df_cc.to_csv('transitivity_aut.csv', index=False)

In [ ]:
#transitivity
#df_cc= pd.DataFrame(columns = ['Date','Author Layer cc','Comment Layer cc','Topic Layer cc'])
df_deg= pd.DataFrame(columns = ['Date','avg_deg'])
#df_cc = pd.read_csv('transitivity.csv')
# assign directory
directory = 'Graphs'
 
# iterate over files in
# that directory

for filename in os.listdir(directory):
    
    f = os.path.join(directory, filename)
    #checking if it is a file
    if os.path.isfile(f):
        print(filename[:11])
        G = nx.read_gexf(f)
 
        deg = average_degree(G)
        new_row = {'Date':filename[:11],'avg_deg':deg}
        
        df_deg.loc[len(df_deg)] = new_row


    df_deg.sort_values(by='Date', inplace = True)
    df_deg = df_deg.reset_index(drop=True)

    df_deg.to_csv('degree.csv', index=False)